In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
import os
import re
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from joblib import dump, load
import ltn
import csv
import math

dataset_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU'

PGB_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU\\PGB'
RGB_path = 'C:\\work_ssd\\DDS_Paper\\data\\DDS_Data_SEU\\RGB'

# Specify the CSV file path
csv_file = 'C:\\work_ssd\\DDS_Paper\\data\\data_robust.csv'
preprocessor_file = 'preprocessor.joblib'


np.random.seed(45)

# Set the chunk size for reading the CSV
chunk_size = 100000  # Adjust the chunk size according to your memory limitations

In [2]:
def extract_fault(file_name):
    match = re.search(r'\d+', file_name)
    if match:
        return int(match.group(0)[0])  # Extract the first digit
    else:
        return None

def process_files_to_csv(data_folders, output_file):
    # Check if the file already exists
    if os.path.isfile(output_file):
        print(f"File {output_file} already exists. Skipping processing.")
        return

    total_files = sum([len(files) for data_folder in data_folders for r, d, files in os.walk(data_folder)])

    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Speed', 'Type', 'Fault'])

        with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
            for data_folder in data_folders:
                for root, dirs, files in os.walk(data_folder):
                    if '.ipynb_checkpoints' in root:
                        continue  # Skip .ipynb_checkpoints folders
                    for file in files:
                        if file.endswith('.txt'):
                            file_path = os.path.join(root, file)
                            path_parts = file_path.split('\\')
                            variable_speed = 'Variable_speed' in file_path
                            type_index = -4 if variable_speed else -3
                            type = path_parts[type_index] if path_parts[type_index] in ['PGB', 'RGB'] else None
                            if type is not None:
                                speed_index = -3 if variable_speed else -2
                                speed = path_parts[speed_index]
                                fault = extract_fault(file)

                                data = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')
                                reshaped_data = data.values[:, :]

                                for row_data in tqdm(reshaped_data, desc="Processing rows", unit="row", leave=False):
                                    row = row_data.tolist() + [speed, type, fault]
                                    csv_writer.writerow(row)
                            pbar.update()

In [3]:
# Example usage
process_files_to_csv([PGB_path, RGB_path], csv_file)

File C:\work_ssd\DDS_Paper\data\data_robust.csv already exists. Skipping processing.


In [ ]:
def split_and_sample_data(file_path, output_train, output_val, train_ratio=0.8, sample_fraction=0.01):
    chunksize = 100000
    total_lines = sum([80740352, 80740352, 80740352, 80740352, 80740352, 80740352, 80740352, 80740352, 80740352])
    total_chunks = math.ceil(total_lines / chunksize)

    reader = pd.read_csv(file_path, chunksize=chunksize)

    with open(output_train, 'w', newline='') as train_file, open(output_val, 'w', newline='') as val_file:
        train_writer = csv.writer(train_file)
        val_writer = csv.writer(val_file)
        
        for i, chunk in tqdm(enumerate(reader), total=total_chunks, desc="Processing chunks", unit="chunk"):
            chunk_sample = chunk.sample(frac=sample_fraction, random_state=1)
            if i == 0:
                train_writer.writerow(chunk_sample.columns.values)
                val_writer.writerow(chunk_sample.columns.values)

            train_data = chunk_sample.iloc[:int(train_ratio*len(chunk_sample))].values
            val_data = chunk_sample.iloc[int(train_ratio*len(chunk_sample)):].values

            train_writer.writerows(train_data)
            val_writer.writerows(val_data)

# example usage
split_and_sample_data(csv_file, 'train.csv', 'val.csv')

# Data preprocessing

In [ ]:

# Initialize a dictionary to store the fault counts
fault_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv('train.csv', chunksize=chunk_size):
        
        #print(chunk)
        # Assuming there is a column named 'fault' in the CSV representing the fault type
        fault_column = 'Fault'

        # Count the occurrences of each fault in the current chunk
        fault_chunk_counts = chunk[fault_column].value_counts()

        # Aggregate the counts with the overall fault_counts dictionary
        for fault, count in fault_chunk_counts.items():
            fault_counts[fault] = fault_counts.get(fault, 0) + count

        pbar.update()

# Print the fault counts
for fault, count in fault_counts.items():
    print(f"Fault: {fault}, Count: {count}")


In [ ]:
# Initialize dictionaries to store the counts
speed_counts = {}
type_counts = {}

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv('train.csv', chunksize=chunk_size):
        # Assuming there is a column named 'Speed' in the CSV representing the speed values
        
        speed_column = 'Speed'

        # Count the occurrences of each speed value in the current chunk
        speed_chunk_counts = chunk[speed_column].value_counts()

        # Aggregate the counts with the overall speed_counts dictionary
        for speed, count in speed_chunk_counts.items():
            speed_counts[speed] = speed_counts.get(speed, 0) + count

        # Assuming there is a column named 'Type' in the CSV representing the types
        type_column = 'Type'

        # Count the occurrences of each type in the current chunk
        type_chunk_counts = chunk[type_column].value_counts()

        # Aggregate the counts with the overall type_counts dictionary
        for typ, count in type_chunk_counts.items():
            type_counts[typ] = type_counts.get(typ, 0) + count

        pbar.update()

# Print the speed counts
print("Speed Counts:")
for speed, count in speed_counts.items():
    print(f"Speed: {speed}, Count: {count}")

# Print the type counts
print("Type Counts:")
for typ, count in type_counts.items():
    print(f"Type: {typ}, Count: {count}")


In [4]:
def data_generator(batch_size, data = csv_file):
    chunksize = batch_size
    for chunk in pd.read_csv(data, chunksize=chunksize):
        # One-hot encode the categorical features
        categorical_features = chunk[categorical_features_columns]
        categorical_features = one_hot_encoder.transform(categorical_features).toarray()
        
        # Concatenate with numerical features
        numerical_features = chunk[numerical_features_columns]
        X = np.concatenate([numerical_features, categorical_features], axis=1)

        sample_size = X.shape[0]
        #print(sample_size)
        time_steps = X.shape[1]
        #print(time_steps)
        input_dimensions = 1

        X_reshaped = X.reshape(sample_size,time_steps,input_dimensions)

        # Extract the labels
        y = chunk['Fault'].values

        yield X_reshaped, y

def data_generator_all(batch_size, data = csv_file):
    # Read all the data
    df = pd.read_csv(data)
    
    # Shuffle the DataFrame rows 
    df = df.sample(frac=1)

    # Calculate the number of batches
    num_batches = len(df) // batch_size

    for i in range(num_batches):
        batch = df.iloc[i*batch_size:(i+1)*batch_size]

        # One-hot encode the categorical features
        categorical_features = batch[categorical_features_columns]
        categorical_features = one_hot_encoder.transform(categorical_features).toarray()

        # Concatenate with numerical features
        numerical_features = batch[numerical_features_columns].values
        X = np.concatenate([numerical_features, categorical_features], axis=1)

        sample_size = X.shape[0]
        time_steps = X.shape[1]
        input_dimensions = 1

        X_reshaped = X.reshape(sample_size, time_steps, input_dimensions)

        # Extract the labels
        y = batch['Fault'].values

        yield X_reshaped, y


# Define the categories
# Define the categories
speed_categories = ['20_0','30_0','30_1','30_2', '30_3','30_4','30_5','40_0','50_0', 'Variable_speed']
type_categories = ['PGB', 'RGB']

# Initialize the OneHotEncoder
one_hot_encoder = OneHotEncoder(categories=[speed_categories, type_categories])

# Create a dummy dataset
dummy_df = pd.DataFrame(data=[['20_0', 'PGB']], columns=['Speed', 'Type'])

# Fit the encoder
one_hot_encoder.fit(dummy_df)

# Define your feature column names
categorical_features_columns = ['Speed', 'Type']
numerical_features_columns = ['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8']

# batch_size = 2  # Adjust as necessary
# dataset = tf.data.Dataset.from_generator(data_generator, args=[batch_size], output_signature=(
#     tf.TensorSpec(shape=(batch_size, 1, 20), dtype=tf.float32),  # Update this to match the shape of X
#     tf.TensorSpec(shape=(batch_size,), dtype=tf.int32)
# ))


In [7]:
generator = data_generator_all(batch_size=256, data = 'train.csv')
last_X, last_y = None, None

for sample_X, sample_y in generator:
    last_X, last_y = sample_X, sample_y
    print(last_X.shape)
    break

print(last_X)
print(last_y)

(256, 20, 1)
[[[-1.44579e-01]
  [ 5.44400e-03]
  [ 8.12200e-03]
  ...
  [ 0.00000e+00]
  [ 0.00000e+00]
  [ 1.00000e+00]]

 [[-1.54454e-01]
  [-8.46000e-04]
  [ 8.25200e-03]
  ...
  [ 0.00000e+00]
  [ 0.00000e+00]
  [ 1.00000e+00]]

 [[-1.26055e-01]
  [ 2.50720e-02]
  [-5.09020e-02]
  ...
  [ 0.00000e+00]
  [ 1.00000e+00]
  [ 0.00000e+00]]

 ...

 [[-1.56269e-01]
  [ 1.63200e-03]
  [-4.66000e-04]
  ...
  [ 0.00000e+00]
  [ 1.00000e+00]
  [ 0.00000e+00]]

 [[-1.38204e-01]
  [-2.42000e-04]
  [-6.48300e-03]
  ...
  [ 0.00000e+00]
  [ 0.00000e+00]
  [ 1.00000e+00]]

 [[-1.30302e-01]
  [-2.14300e-03]
  [ 7.10200e-03]
  ...
  [ 0.00000e+00]
  [ 0.00000e+00]
  [ 1.00000e+00]]]
[8 1 7 4 7 4 2 3 6 2 8 3 1 6 3 2 7 7 7 8 2 5 2 4 8 7 4 2 2 1 0 1 8 1 8 4 5
 4 8 5 3 8 4 6 4 0 7 0 4 3 0 7 1 8 7 1 0 8 2 7 5 8 6 8 1 5 4 2 1 3 2 1 6 8
 2 6 6 3 7 1 7 3 0 8 0 7 6 1 1 4 8 3 0 3 8 7 3 5 2 7 7 5 1 7 3 1 1 8 5 8 5
 0 7 0 0 5 8 2 6 5 1 8 4 4 7 2 4 0 2 5 3 8 6 0 7 2 4 2 1 4 6 6 8 7 3 6 3 3
 1 3 8 5 7 0 7 4 2 3 

## CNN as backbone

In [6]:
import keras

def build_cnn_backbone():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(last_X.shape[1],last_X.shape[2])))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=4, activation='elu', name="Conv1D_1"))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Conv1D(filters=32, kernel_size=3, activation='elu', name="Conv1D_2"))
    model.add(keras.layers.Conv1D(filters=16, kernel_size=2, activation='elu', name="Conv1D_3"))
    model.add(keras.layers.MaxPooling1D(pool_size=2, name="MaxPooling1D"))
    model.add(keras.layers.Flatten())
    #model.add(keras.layers.Dense(8, activation='relu', name="Dense_1"))
    model.add(keras.layers.Dense(9, name="Dense_2"))
    return model

# Build the CNN backbone model
model = build_cnn_backbone()

# Wrap the model in an LTN predicate
p = ltn.Predicate.FromLogits(model, activation_function="softmax", with_class_indexing=True)


In [8]:
# Constants to index/iterate on the classes
HEALTHY = ltn.Constant(0, trainable=False)
CTF = ltn.Constant(1, trainable=False)
MTF = ltn.Constant(2, trainable=False)
RCF = ltn.Constant(3, trainable=False)
SWF = ltn.Constant(4, trainable=False)
BWF = ltn.Constant(5, trainable=False)
CWF = ltn.Constant(6, trainable=False)
IRF = ltn.Constant(7, trainable=False)
ORF = ltn.Constant(8, trainable=False)

In [9]:
#operators
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [10]:
# Define the formula aggregator
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    # Variables for each class
    x_healthy = ltn.Variable("x_healthy", features[labels == 0])
    x_ctf = ltn.Variable("x_ctf", features[labels == 1])
    x_mtf = ltn.Variable("x_mtf", features[labels == 2])
    x_rcf = ltn.Variable("x_rcf", features[labels == 3])
    x_swf = ltn.Variable("x_swf", features[labels == 4])
    x_bwf = ltn.Variable("x_bwf", features[labels == 5])
    x_cwf = ltn.Variable("x_cwf", features[labels == 6])
    x_irf = ltn.Variable("x_irf", features[labels == 7])
    x_orf = ltn.Variable("x_orf", features[labels == 8])

    # Fault list for mutual exclusivity axioms
    faults = [HEALTHY, CTF, MTF, RCF, SWF, BWF, CWF, IRF, ORF]
    fault_vars = [x_healthy, x_ctf, x_mtf, x_rcf, x_swf, x_bwf, x_cwf, x_irf, x_orf]

    axioms = []
    for i, fault in enumerate(faults):
        # Add the axiom that for all x of a certain fault, the probability of that fault should be high
        axioms.append(Forall(fault_vars[i], p([fault_vars[i], fault], training=training)))

        # Add the axioms for mutual exclusivity
        for j, other_fault in enumerate(faults):
            if i != j:
                axioms.append(Forall(fault_vars[i], Not(p([fault_vars[i], other_fault], training=training))))

    sat_level = formula_aggregator(axioms).tensor
    return sat_level


In [11]:

# Print the initial satisfaction level for each batch of the test dataset
for sample_X, sample_y in generator:
    #sample_X_reshaped = tf.reshape(sample_X, (sample_X.shape[0], sample_X.shape[1], 1))
    #print(sample_X)
    print("Initial sat level %.5f" % axioms(sample_X, sample_y))
    break

Initial sat level 0.68522


In [12]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}


In [13]:

@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = model([features])
    metrics_dict['train_accuracy'](labels, predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = model([features])
    metrics_dict['test_accuracy'](labels, predictions)


In [14]:
from collections import defaultdict
import time
from tqdm import tqdm

# Revised training function with tqdm progress tracking
def train(epochs, metrics_dict, train_generator_func, test_generator_func, train_step, test_step,
          num_train_steps, num_test_steps, track_metrics=1, csv_path=None, scheduled_parameters=defaultdict(lambda: {})):

    template = "Epoch {}"
    for metrics_label in metrics_dict.keys():
        template += ", %s: {:.4f}" % metrics_label

    if csv_path is not None:
        csv_file = open(csv_path, "w+")
        headers = ",".join(["Epoch"] + list(metrics_dict.keys()))
        csv_template = ",".join(["{}" for _ in range(len(metrics_dict) + 1)])
        csv_file.write(headers + "\n")

    epoch_times = []
    start_time = time.time()

    for epoch in range(epochs):
        print(f"Starting epoch {epoch}")
        for metrics in metrics_dict.values():
            metrics.reset_states()

        # Reset the generators for each epoch
        train_generator = train_generator_func()

        print(f"Starting training for epoch {epoch}")
        pbar = tqdm(total=num_train_steps)
        for batch_elements in train_generator:
            train_step(*batch_elements, **scheduled_parameters[epoch])
            pbar.set_description(f"Training sat: {metrics_dict['train_sat_kb'].result():.4f}")
            pbar.update()
        pbar.close()

        test_generator = test_generator_func()

        print(f"Starting testing for epoch {epoch}")
        pbar = tqdm(total=num_test_steps)
        for batch_elements in test_generator:
            test_step(*batch_elements, **scheduled_parameters[epoch])
            pbar.set_description(f"Testing sat: {metrics_dict['test_sat_kb'].result():.4f}")
            pbar.update()
        pbar.close()

        metrics_results = [metrics.result() for metrics in metrics_dict.values()]
        if epoch % track_metrics == 0:
            print(template.format(epoch, *metrics_results))

        if csv_path is not None:
            csv_file.write(csv_template.format(epoch, *metrics_results) + "\n")
            csv_file.flush()

        end_time = time.time()
        epoch_time = end_time - start_time
        start_time = end_time
        epoch_times.append(epoch_time)

        print(f"Epoch {epoch} completed. Time: {epoch_time}")

    if csv_path is not None:
        csv_file.close()

    return epoch_times


In [15]:
batch_size = 256

EPOCHS = 5

# Get the number of training and testing steps per epoch
num_train_steps = sum(1 for _ in data_generator(batch_size, data='train.csv'))
num_test_steps = sum(1 for _ in data_generator(batch_size, data='val.csv'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define your generator functions
def train_generator_func():
    return data_generator_all(batch_size=batch_size, data='train.csv')

def val_generator_func():
    return data_generator_all(batch_size=batch_size, data='val.csv')



In [16]:

training = train(
    EPOCHS,
    metrics_dict,
    train_generator_func,
    val_generator_func,
    train_step,
    test_step,
    csv_path="final.csv",
    track_metrics=1,
    num_train_steps=num_train_steps,
    num_test_steps=num_test_steps
)

Starting epoch 0
Starting training for epoch 0


Training sat: 0.6955: 100%|█████████▉| 22708/22709 [40:25<00:00,  9.36it/s]


Starting testing for epoch 0


Testing sat: 0.6977: 100%|█████████▉| 5677/5678 [02:10<00:00, 43.65it/s]


Epoch 0, train_sat_kb: 0.6955, test_sat_kb: 0.6977, train_accuracy: 0.0696, test_accuracy: 0.0708
Epoch 0 completed. Time: 2555.9184296131134
Starting epoch 1
Starting training for epoch 1


...

Starting testing for epoch 2


Testing sat: 0.6991: 100%|█████████▉| 5677/5678 [01:59<00:00, 47.68it/s]


Epoch 2, train_sat_kb: 0.6991, test_sat_kb: 0.6991, train_accuracy: 0.0708, test_accuracy: 0.0710
Epoch 2 completed. Time: 2465.9900348186493
Starting epoch 3
Starting training for epoch 3


Training sat: 0.6993:  42%|████▏     | 9631/22709 [16:14<22:04,  9.87it/s]

KeyboardInterrupt: 